https://github.com/dtrckd/simplon_datai_2020/blob/master/brief_11/brief.md

veille lstm

https://docs.google.com/document/d/1teI_eKv4zVdJy23-ZfK5n0tvRmklPgXf1gl4w6B8Ot4/edit

https://penseeartificielle.fr/comprendre-lstm-gru-fonctionnement-schema/



    Get the the dataset and give an overview of the data
    Show a graphics of the Close prices (you can use plotly to have zoom capability)
    ordonné le dataset en fonction de la date
    decode date value using the fonction pd.to_datetime. Explain the purpose of this step ?
    normalize the dataset (You can use the MinMaxScaler). Explain the purpose of this step ?

# Model 1

Objectif: to predict the last 250 days prices given all the previous observation

Build a train and a test set in order to evaluate the model performance.

    Utiliser une fenetre d'observation de 50 jours
    Utiliser un sequence de prédiction de 1



Obtenez l'ensemble de données et donnez un aperçu des données
Afficher un graphique des prix de clôture (vous pouvez utiliser l'intrigue pour avoir la capacité de zoom)
ordonné le dataset en fonction de la date
décoder la valeur de la date à l'aide de la fonction pd.to_datetime. Expliquez le but de cette étape?
normaliser l'ensemble de données (vous pouvez utiliser le MinMaxScaler). Expliquez le but de cette étape?

pour prédire les prix des 250 derniers jours compte tenu de toutes les observations précédentes

Construisez un train et un ensemble de test afin d'évaluer les performances du modèle.

In [1]:
import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
from plotly.offline import init_notebook_mode, iplot, plot
import plotly as py
init_notebook_mode(connected=True)
import plotly.graph_objs as go
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout
from keras.models import Sequential
from keras import layers
from sklearn.linear_model import LinearRegression

## 1-Obtenez l'ensemble de données et donnez un aperçu des données

In [2]:
data = pd.read_csv("stock_price.csv",usecols=['Date','Close'])

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1235 entries, 0 to 1234
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    1235 non-null   object 
 1   Close   1235 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.4+ KB


In [4]:
data.head()

,Date,Close
0,2018-10-08,215.15
1,2018-10-05,209.20
2,2018-10-04,218.20
3,2018-10-03,227.60
4,2018-10-01,230.90


## 2-Afficher un graphique des prix de clôture

In [5]:

trace1 = go.Scatter(
                    x = data.Date,
                    y = data.Close,
                    mode = "lines",
                    name = "citations",
                    marker = dict(color = 'rgba(16, 112, 2, 0.8)'),
                    text = data)
    
    

layout = dict(title = 'Prix de cloture par date',
              xaxis = dict(title = 'Date',ticklen = 5,zeroline= False),
              yaxis = dict(title = 'Prix  de cloture')
             )
fig = dict(data = trace1, layout = layout)
iplot(fig)

## 3-ordonner le dataset en fonction de la date

In [6]:
data.sort_values(by='Date', ascending=True, inplace=True)
data.head()
#data.reset_index(drop=True,inplace=True) #reset les index et ne crée pas de nouvelle colonne

,Date,Close
1234,2013-10-08,155.80
1233,2013-10-09,155.55
1232,2013-10-10,160.15
1231,2013-10-11,160.05
1230,2013-10-14,159.45


## 4-décoder la valeur de la date à l'aide de la fonction pd.to_datetime. Expliquez le but de cette étape?

In [7]:
pd.to_datetime(data['Date'],errors='coerce') 
data['Date'].str.replace("-","").astype(int) 




#data['Date'].str.replace("-","").astype(int) 
#pd.to_datetime((data['Date']), format=YYYYMMDD) 
#data['Date'].dt.strftime("YYYYMMDD").astype(int)
#pd.to_datetime(data["Date"]).datetime.strftime("%Y%m%d")
#df = pd.read_csv("stock_price.csv"(parse_dates= [col], encoding='utf-8-sig', usecols= ['Date']))    
#df['Month'] = df['Date'].dt.month
#df['Date'].str.replace("-","").astype(int) 
#df['Date'].dt.strftime("%Y%m%d").astype(int)

1234    20131008
1233    20131009
1232    20131010
1231    20131011
1230    20131014
          ...   
4       20181001
3       20181003
2       20181004
1       20181005
0       20181008
Name: Date, Length: 1235, dtype: int64

## 5- normaliser l'ensemble de données (vous pouvez utiliser le MinMaxScaler). Expliquez le but de cette étape?

In [8]:
X = data['Close']
scaler = MinMaxScaler()
close_reshape = np.array(X).reshape(-1, 1)


close_fit_tr = scaler.fit_transform(close_reshape)


print(close_fit_tr)
print(close_reshape.shape)
print(scaler.data_max_)
print(scaler.data_min_)



[[0.23823398]
 [0.2371134 ]
 [0.25773196]
 ...
 [0.51792918]
 [0.47758853]
 [0.50425818]]
(1235, 1)
[325.75]
[102.65]


In [9]:
len(close_reshape)

1235

## 6- définition train et test

In [10]:
X_train = []
y_train = []
len_seq = 50
len_test = 250

for i in range(len_seq, len(close_reshape)-len_test): 
    X_train.append(close_fit_tr[i-50:i,0])  #close_fit_tr correspond au vrai prix normalisé
    y_train.append(close_fit_tr[i,0])
    
X_train, y_train =np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

#range nous sort une liste de nombre partant de len_seq donc 50 jusqu'a len(close_reshape)-len_test
#ensuite on prend pour la premiere sequence le X_train, i-50:i,0  c'est a dire de zero(i-50) à 49(:i) colonne zero.
#on ajoute ca avec le append dans la list vide definit plus haut
#pour le y_train on prend une seule valeur celle que l'on veut predire et qui suit la sequence definit
#donc pour la premiere x t_train de zero à 49 et y_train est la ligne 50
#interet de la boucle decale chaque x_train et chaque y_train de 1
#ex sequence suivante va commencer a 1 finir a 50 et y sera 51 et ainsi dessuite.

#i commence à 50 jusqu'a 985

In [12]:
X_test = []
y_test = []
len_seq = 50
len_test = 250

for i in range(len(close_reshape)-len_test, len(close_reshape)): 
    X_test.append(close_fit_tr[i-50:i,0])  #close_fit_tr correspond au vrai prix normalisé
    y_test.append(close_fit_tr[i,0])
    
X_test, y_test =np.array(X_test), np.array(y_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

#i commence à 985 jusqua la fin
#la premiere sequence du x_test correspond a la derniere du X_train

## 7-Définition du modèle

In [13]:

model = Sequential()
model.add(LSTM(50,return_sequences = True, input_shape=( X_train.shape[1], 1)))
model.add(Dropout(0.1))

model.add(LSTM(50,return_sequences = True))
model.add(Dropout(0.1))

model.add(LSTM(50,return_sequences = True))
model.add(Dropout(0.1))

model.add(LSTM(50))
model.add(Dropout(0.1))

model.add(layers.Dense(1, activation='linear'))
model.compile(optimizer='adam', loss='mse')




model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50, 50)            10400     
_________________________________________________________________
dropout (Dropout)            (None, 50, 50)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 50)            20200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 50)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 50, 50)            20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 50, 50)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                2

In [14]:
history = model.fit(X_train, y_train, epochs = 50, verbose =1)

Epoch 1/50
30/30 [==============================] - 4s 123ms/step - loss: 0.0069
Epoch 2/50
30/30 [==============================] - 3s 115ms/step - loss: 0.0019
Epoch 3/50
30/30 [==============================] - 3s 106ms/step - loss: 0.0015
Epoch 4/50
30/30 [==============================] - 3s 106ms/step - loss: 0.0015
Epoch 5/50
30/30 [==============================] - 3s 103ms/step - loss: 0.0013
Epoch 6/50
30/30 [==============================] - 3s 103ms/step - loss: 0.0013
Epoch 7/50
30/30 [==============================] - 3s 105ms/step - loss: 0.0013
Epoch 8/50
30/30 [==============================] - 3s 103ms/step - loss: 0.0012
Epoch 9/50
30/30 [==============================] - 3s 104ms/step - loss: 0.0012
Epoch 10/50
30/30 [==============================] - 3s 116ms/step - loss: 0.0012
Epoch 11/50
30/30 [==============================] - 4s 118ms/step - loss: 0.0012
Epoch 12/50
30/30 [==============================] - 3s 111ms/step - loss: 0.0013
Epoch 13/50
30/30 [======

## 8-Prédiction

In [15]:
y_pred= model.predict(X_test)
y_pred

array([[0.46478093],
       [0.46621913],
       [0.4706189 ],
       [0.47171253],
       [0.47059703],
       [0.4715742 ],
       [0.47873127],
       [0.4830305 ],
       [0.4817719 ],
       [0.48001027],
       [0.47357994],
       [0.46356505],
       [0.45427775],
       [0.4615124 ],
       [0.46739912],
       [0.474509  ],
       [0.49884284],
       [0.5329257 ],
       [0.55257666],
       [0.5548014 ],
       [0.5467707 ],
       [0.5385888 ],
       [0.55528504],
       [0.5589165 ],
       [0.55035466],
       [0.5747872 ],
       [0.5971183 ],
       [0.59751976],
       [0.61317056],
       [0.6399263 ],
       [0.6568699 ],
       [0.67638975],
       [0.72240394],
       [0.7482409 ],
       [0.74947244],
       [0.74642694],
       [0.75853044],
       [0.76958096],
       [0.800673  ],
       [0.8019735 ],
       [0.8081817 ],
       [0.80714774],
       [0.79334885],
       [0.78922665],
       [0.78204757],
       [0.8024503 ],
       [0.83253044],
       [0.868

In [16]:
y_pred_invers = scaler.inverse_transform(y_pred) #renvoie les vrai prix avant normalisation
print(y_pred_invers.shape)
pred_price = y_pred_invers.ravel() #aplanit pareil que .reshape
print(pred_price.shape)

(250, 1)
(250,)


In [17]:
vrai_prix = data.Close[len(data)-len_test:].values #renvoi les valeurs de 'close' -les 250 du test
#pred_price = scaler.inverse_transform(y_pred)
time  = data.Date[len(data)-len_test:].values #renvoi les valeur de 'date' - les 250 du test

In [18]:
pred_price[6]

209.45494

In [19]:
vrai_prix.shape

(250,)

## 9-Représentation

In [22]:
def plt_result(time,vrai_prix,pred_price):
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=time,
            y=vrai_prix,
            name = 'Vrai Prix'))
    fig.add_trace(
        go.Scatter(
            x=time,
            y=pred_price,
            name='Prix Prédit'))
    fig.update_layout(
        title='stock price prediction',
        xaxis_title_text='time(day)',
        yaxis_title_text='Price')
    return fig
    

In [23]:
fig = plt_result(time,vrai_prix,pred_price)
fig.show()